In [1]:
# Load libraries
import os 
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as st
import missingno as msno

In [2]:
extension = 'csv'
path = r'C:\Users\Justin\Data Science\NJ Transit\data'
all_files = glob.glob(path + "/*.csv")

li = []

In [3]:
for file in all_files:
    df = pd.read_csv(file, index_col=None)
    li.append(df)
    
#frame = pd.concat(li, axis=0, ignore_index=True)
#frame = pd.read_csv('data/2018_03.csv', index_col=None)
#frame2 = pd.read_csv('data/2018_04.csv', index_col=None)
#frame3 = pd.read_csv('data/2018_05.csv', index_col=None)

In [4]:
frame.head(20)

NameError: name 'frame' is not defined

In [ ]:
frame.tail()

In [ ]:
frame.describe()

In [ ]:
frame.shape

In [ ]:
numeric_features = frame.select_dtypes(include=[np.number])
numeric_features.columns

In [ ]:
categorical_features = frame.select_dtypes(include=[np.object])
categorical_features.columns

In [ ]:
msno.matrix(frame.sample(1000))

In [ ]:
msno.bar(frame.sample(1000))

In [ ]:
msno.dendrogram(frame)

In [ ]:
frame.skew()

In [ ]:
frame.kurt()

In [ ]:
# Remove rows where we have no delay_minutes value
frame = frame.dropna(subset=['delay_minutes'])

In [ ]:
y = frame['delay_minutes']

plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)

plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)

plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)

In [ ]:
sns.distplot(frame.skew(), color='blue', axlabel='Skewness')

In [ ]:
plt.figure(figsize = (12,8))
sns.distplot(frame.kurt(), color='r', axlabel='Kurtosis', norm_hist=False, kde=True, rug=False)
plt.show()

In [ ]:
plt.hist(frame['delay_minutes'], orientation='vertical', histtype='bar', color='blue')
plt.show()

In [ ]:
correlation = numeric_features.corr()
print(correlation['delay_minutes'].sort_values(ascending=False), '\n')

In [ ]:
f, ax = plt.subplots(figsize = (15, 15))
sns.heatmap(correlation, square=True, linewidths=0.5, annot=True)
plt.show()

In [ ]:
sns.set()
columns = ['delay_minutes', 'stop_sequence', 'to_id', 'from_id']
sns.pairplot(frame[numeric_features.columns], size=2, kind='scatter', diag_kind='kde')
plt.show()

In [ ]:
fig, ((ax1, ax2, ax3)) = plt.subplots(nrows=1, ncols=3, figsize=(14,10))

stop_sequence_scatter = pd.concat([frame['delay_minutes'], frame['stop_sequence']], axis=1)
sns.regplot(x='stop_sequence', y='delay_minutes', data=stop_sequence_scatter, scatter=True, fit_reg=True, ax=ax1)

from_id_scatter = pd.concat([frame['delay_minutes'], frame['from_id']], axis=1)
sns.regplot(x='from_id', y='delay_minutes', data=from_id_scatter, scatter=True, fit_reg=True, ax=ax2)

to_id_scatter = pd.concat([frame['delay_minutes'], frame['to_id']], axis=1)
sns.regplot(x='to_id', y='delay_minutes', data=to_id_scatter, scatter=True, fit_reg=True, ax=ax3)

In [ ]:
delay_minutes_stop_sequence = frame.pivot_table(index='stop_sequence', values='delay_minutes', aggfunc=np.median)
delay_minutes_stop_sequence.plot(kind='bar', color='blue')
plt.xlabel('stop_sequence')
plt.ylabel('median delay minutes')
plt.show()

In [ ]:
var = 'stop_sequence'
data = pd.concat([frame['delay_minutes'], frame[var]], axis=1)
f, ax = plt.subplots(figsize=(12,8))
fig = sns.boxplot(x=var, y='delay_minutes', data=data)
fig.axis(ymin=0, ymax=150)

In [ ]:
var = 'line'
data = pd.concat([frame['delay_minutes'], frame[var]], axis=1)
f, ax = plt.subplots(figsize=(12,8))
fig = sns.boxplot(x=var, y='delay_minutes', data=data)
fig.axis(ymin=0, ymax=20);
xt = plt.xticks(rotation=45)

In [ ]:
var = 'from_id'
data = pd.concat([frame['delay_minutes'], frame[var]], axis=1)
f, ax = plt.subplots(figsize=(12,8))
fig = sns.boxplot(x=var, y='delay_minutes', data=data)
fig.axis(ymin=0, ymax=20);
xt = plt.xticks(rotation=90)

In [ ]:
# see how date effects delay_minutes
frame['date'] = pd.to_datetime(frame['date'])
plt.plot_date(x=frame['date'], y=frame['delay_minutes'], fmt='r-')
plt.xticks(rotation=90)

In [ ]:
no_time_features = frame[['status', 'line', 'type']]
no_time_features.columns

In [ ]:
for c in no_time_features:
    frame[c] = frame[c].astype('category')
    if frame[c].isnull().any():
        frame[c] = frame[c].cat.add_categories(['MISSING'])
        frame[c] = frame[c].fillna('MISSING')
        
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
    
f = pd.melt(frame, id_vars=['delay_minutes'], value_vars=no_time_features)
g = sns.FacetGrid(f, col='variable', col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, 'value', 'delay_minutes')

In [ ]:
frame['long_delay'] = frame['delay_minutes'] > 5
frame.groupby('line')['long_delay'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
x= frame.groupby(['line', 'status']).size().unstack()
x['cancelled']/(x['departed']+x['estimated'])

In [ ]:
x['cancelled']

In [ ]:
ax = frame.groupby('stop_sequence')['delay_minutes'].mean().plot()
ax.set_ylabel('average delay_minutes')

In [ ]:
frame.groupby('from')['delay_minutes'].mean().sort_values(ascending=False).head(10)

In [ ]:
frame.groupby('to')['delay_minutes'].mean().sort_values(ascending=False).head(10)

In [ ]:
x = frame.groupby('date')['delay_minutes'].mean()
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
fig.autofmt_xdate()
ax.plot(x)
ax.set_ylabel('average delay_minutes')
plt.show()

In [ ]:
frame.scheduled_time = pd.to_datetime(frame.scheduled_time)
frame['time'] = frame.scheduled_time.dt.time

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 8)
x = frame.groupby('time')['delay_minutes'].mean()
ax.plot(x)
ax.set_ylabel('average delay_minutes')
plt.show()

In [ ]:
x.sort_values(ascending=False).head(5)

In [ ]:
x = frame.groupby(['date', 'status']).size().unstack()
x = (x['cancelled']/(x['departed']+x['estimated']))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 8)
ax.plot(x)
ax.set_ylabel('cancellation rate')
plt.show()

In [ ]:
x.sort_values(ascending=False).head(5)

In [ ]:
x = frame.groupby(['time', 'status']).size().unstack()
x = (x['cancelled']/(x['departed']+x['estimated'])) 

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.plot(x)
ax.set_ylabel('cancellation rate')
plt.show()